In [1]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the trained model
model = joblib.load("chronic_model.pkl")


In [2]:
# Inputs for the model
highbp = widgets.Dropdown(options=[0, 1], description='HighBP:')
highchol = widgets.Dropdown(options=[0, 1], description='HighChol:')
bmi = widgets.FloatText(value=25.0, description='BMI:')
smoker = widgets.Dropdown(options=[0, 1], description='Smoker:')
phys_activity = widgets.Dropdown(options=[0, 1], description='PhysActivity:')
fruits = widgets.Dropdown(options=[0, 1], description='Fruits:')
veggies = widgets.Dropdown(options=[0, 1], description='Veggies:')
age = widgets.IntSlider(value=40, min=0, max=100, step=1, description='Age:')



In [3]:
def recommend_tips(bmi, smoker, fruits, phys_activity):
    tips = []
    if bmi > 30:
        tips.append("Try reducing your BMI through diet and daily walks.")
    if smoker == 1:
        tips.append("Consider quitting smoking to lower chronic disease risk.")
    if fruits == 0:
        tips.append("Eat at least 1 fruit daily.")
    if phys_activity == 0:
        tips.append("Start light physical activity for 20 minutes daily.")
    return "💡 " + " ".join(tips) if tips else "💡 You're doing great! Keep it up!"


In [4]:
def show_bmi_chart(bmi_value):
    fig, ax = plt.subplots()
    ax.bar(["Your BMI"], [bmi_value], color='red' if bmi_value > 30 else 'green')
    ax.axhline(25, linestyle='--', color='gray', label='Normal Threshold')
    ax.set_title("BMI Level")
    ax.legend()
    plt.show()


In [5]:
# Prediction button and output display
predict_button = widgets.Button(description='Predict Risk')
output = widgets.Output()

def on_predict_click(b):
    lifestyle_score = phys_activity.value + (1 - smoker.value)
    diet_score = fruits.value + veggies.value

    input_data = pd.DataFrame([[ 
        highbp.value, highchol.value, bmi.value, age.value,
        lifestyle_score, diet_score
    ]], columns=['HighBP', 'HighChol', 'BMI', 'Age', 'lifestyle_score', 'diet_score'])

    prediction = model.predict(input_data)[0]

    # Save to CSV log
    input_data["Prediction"] = prediction
    input_data["Timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    os.makedirs("logs", exist_ok=True)
    log_file = "logs/prediction_log.csv"
    input_data.to_csv(log_file, mode="a", header=not os.path.exists(log_file), index=False)

    # Display prediction and tips
    with output:
        output.clear_output()
        if prediction == 1:
            print("⚠️ You are at risk for Diabetes. Consider lifestyle changes and consult a doctor.")
        else:
            print("✅ You are not at high risk. Maintain your healthy lifestyle.")
        print(recommend_tips(bmi.value, smoker.value, fruits.value, phys_activity.value))
        show_bmi_chart(bmi.value)

predict_button.on_click(on_predict_click)


In [6]:
input_form = widgets.VBox([
    highbp, highchol, bmi, smoker,
    phys_activity, fruits, veggies, age,
    predict_button
])

display(widgets.VBox([input_form, output]))
